# Data Loading and Inspection

In [5]:
# ==========================================
# Cell 1) Detection datasets discovery & inspection (FINAL)
#   - How many datasets are found
#   - train/val image count per dataset
#   - Whether label cases (original/scale/side) exist
#   - Output estimated class count/names (multiclass-based)
#
# [UPDATED]
#   ✅ Reflect split structure rules per dataset name
#     * BCCD, brain-tumor, Custom_Blood, homeobjects-3K, kitti, medical-pills, signature
#         images/labels -> train, val
#     * coco
#         images/labels -> train2017, val2017, test2017
#     * construction-ppe, african-wildlife
#         images/labels -> train, val, test
#     * lvis
#         images/labels -> train2017, val2017, test2017
#     * SKU-110K
#         images/labels -> no subfolders
#         -> seed-based 8:2 virtual split
#     * VOC
#         images/labels -> test2007, train2007, train2012, val2007, val2012
#         -> use train2012 / val2012 only
# ==========================================

from __future__ import annotations

import os, sys, random, shutil
from pathlib import Path
from typing import List, Tuple, Optional, Dict

# -------------------------------------------------------------------------
# 0) Register PROJECT_MODULE_DIR
# -------------------------------------------------------------------------
PROJECT_MODULE_DIR = Path("/home/ISW/project/Project_Module")
if str(PROJECT_MODULE_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_MODULE_DIR))

# -------------------------------------------------------------------------
# 1) ultra_det_loader
# -------------------------------------------------------------------------
from ultra_det_loader import discover_det_datasets

# -------------------------------------------------------------------------
# 2) noisy_insection (use only scale/boundary jitter case list)
# -------------------------------------------------------------------------
try:
    from noisy_insection import UNIFORM_SCALING_FACTORS, JITTER_PATTERNS
except Exception:
    UNIFORM_SCALING_FACTORS = [0.6, 0.8, 1.2, 1.4]
    JITTER_PATTERNS = [3, 5, 7]

# -------------------------------------------------------------------------
# User config
# -------------------------------------------------------------------------
LOAD_DIR = "/home/ISW/project/datasets"
SEED = 42

# Image extensions
_IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

def set_seed(seed: int = 42):
    random.seed(seed)

def list_images(dir_path: Optional[Path]) -> List[Path]:
    if dir_path is None or not Path(dir_path).exists():
        return []
    dir_path = Path(dir_path)
    imgs = []
    for p in dir_path.rglob("*"):
        if p.is_file() and p.suffix.lower() in _IMG_EXTS:
            imgs.append(p)
    return sorted(imgs)

def normalize_name(name: str) -> str:
    # Simple normalization to absorb case/separator differences
    n = name.strip().lower()
    n = n.replace("_", "-")
    n = n.replace(" ", "-")
    return n

# -------------------------------------------------------------------------
# Legacy heuristic (fallback)
# -------------------------------------------------------------------------
def _fallback_train_dir(images_root: Path) -> Path:
    if (images_root / "train").is_dir():
        return images_root / "train"
    return images_root

def _fallback_val_dir(images_root: Path) -> Optional[Path]:
    if (images_root / "val").is_dir():
        return images_root / "val"
    if (images_root / "valid").is_dir():
        return images_root / "valid"
    return None

# -------------------------------------------------------------------------
# ✅ Dataset-specific split rules
# -------------------------------------------------------------------------
_SIMPLE_TRAIN_VAL = {
    "bccd",
    "brain-tumor",
    "custom-blood",
    "homeobjects-3k",
    "kitti",
    "medical-pills",
    "signature",
}

_TRAIN_TEST_VAL = {
    "construction-ppe",
    "african-wildlife",
}

def detect_split_dirs(ds_root: Path) -> Dict[str, Optional[Path]]:
    """
    Interpret images/labels split structure based on ds_root.
    Returns:
        {
          "train_img_dir": Path|None,
          "val_img_dir": Path|None,
          "test_img_dir": Path|None,
          "split_mode": str,  # "explicit" | "sku_virtual_8_2" | "fallback"
          "train_tag": str,
          "val_tag": str,
        }
    """
    ds_name = normalize_name(ds_root.name)
    images_root = ds_root / "images"

    # 1) VOC rule: use train2012/val2012 only
    if ds_name == "voc":
        return dict(
            train_img_dir=images_root / "train2012",
            val_img_dir=images_root / "val2012",
            test_img_dir=None,
            split_mode="explicit",
            train_tag="train2012",
            val_tag="val2012",
        )

    # 2) COCO/LVIS rule
    #    - Handle exact coco/lvis names or potential inclusion
    if ds_name == "coco" or "coco" in ds_name:
        return dict(
            train_img_dir=images_root / "train2017",
            val_img_dir=images_root / "val2017",
            test_img_dir=images_root / "test2017",
            split_mode="explicit",
            train_tag="train2017",
            val_tag="val2017",
        )

    if ds_name == "lvis" or "lvis" in ds_name:
        return dict(
            train_img_dir=images_root / "train2017",
            val_img_dir=images_root / "val2017",
            test_img_dir=images_root / "test2017",
            split_mode="explicit",
            train_tag="train2017",
            val_tag="val2017",
        )

    # 3) Explicit train/val structure
    if ds_name in _SIMPLE_TRAIN_VAL:
        return dict(
            train_img_dir=images_root / "train",
            val_img_dir=images_root / "val",
            test_img_dir=None,
            split_mode="explicit",
            train_tag="train",
            val_tag="val",
        )

    # 4) train/test/val structure (only train/val used for summary)
    if ds_name in _TRAIN_TEST_VAL:
        return dict(
            train_img_dir=images_root / "train",
            val_img_dir=images_root / "val",
            test_img_dir=images_root / "test",
            split_mode="explicit",
            train_tag="train",
            val_tag="val",
        )

    # 5) SKU-110K: no subfolders -> virtual split
    #    - Handle name variations
    if ds_name in {"sku-110k", "sku110k", "sku_110k"} or "sku" in ds_name and "110k" in ds_name:
        return dict(
            train_img_dir=images_root,  # Same physical folder since virtual split
            val_img_dir=images_root,
            test_img_dir=None,
            split_mode="sku_virtual_8_2",
            train_tag="virtual_8_2",
            val_tag="virtual_8_2",
        )

    # 6) fallback
    tr = _fallback_train_dir(images_root)
    va = _fallback_val_dir(images_root)
    return dict(
        train_img_dir=tr,
        val_img_dir=va,
        test_img_dir=None,
        split_mode="fallback",
        train_tag=tr.name if tr else "unknown",
        val_tag=va.name if va else "missing",
    )

# -------------------------------------------------------------------------
# Class name estimation
# -------------------------------------------------------------------------
def infer_class_names_from_labels(label_root: Path, max_files: int = 2000) -> List[str]:
    if label_root is None or not label_root.exists():
        return ["class_0"]

    txts = list(label_root.rglob("*.txt"))
    if not txts:
        return ["class_0"]

    txts = txts[:max_files]
    cls_ids = set()

    for t in txts:
        try:
            with open(t, "r", encoding="utf-8") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    cid = int(float(parts[0]))
                    cls_ids.add(cid)
        except Exception:
            continue

    if not cls_ids:
        return ["class_0"]

    max_id = max(cls_ids)
    return [f"class_{i}" for i in range(max_id + 1)]

# -------------------------------------------------------------------------
# Label case detection
# -------------------------------------------------------------------------
def list_label_cases_for_dataset(ds_root: Path) -> List[Tuple[str, str]]:
    cases: List[Tuple[str, str]] = []

    if (ds_root / "labels").is_dir():
        cases.append(("original", "labels"))

    for s in UNIFORM_SCALING_FACTORS:
        d = f"labels_uniform_scaling_{s}"
        if (ds_root / d).is_dir():
            cases.append((f"scale_{s}", d))

    for k in JITTER_PATTERNS:
        d = f"labels_boundary_jitter_{k}"
        if (ds_root / d).is_dir():
            cases.append((f"side_{k}", d))

    return cases

# -------------------------------------------------------------------------
# SKU-110K virtual split count
# -------------------------------------------------------------------------
def compute_sku_virtual_counts(images_root: Path, seed: int = 42, ratio: float = 0.8) -> Tuple[int, int]:
    imgs = list_images(images_root)
    n = len(imgs)
    if n == 0:
        return 0, 0
    rnd = random.Random(seed)
    idxs = list(range(n))
    rnd.shuffle(idxs)
    cut = int(n * ratio)
    n_train = cut
    n_val = n - cut
    return n_train, n_val

# -------------------------------------------------------------------------
# Discover dataset roots
# -------------------------------------------------------------------------
set_seed(SEED)

specs = discover_det_datasets(LOAD_DIR)
roots: List[Path] = []
for s in specs:
    r = Path(s.root)
    if r not in roots:
        roots.append(r)

print("=" * 80)
print(f"[DISCOVERY] Found {len(roots)} unique dataset roots under: {Path(LOAD_DIR).resolve()}")
print("=" * 80)

# -------------------------------------------------------------------------
# Per-dataset summary
# -------------------------------------------------------------------------
dataset_summaries: List[Dict] = []

for ds_root in roots:
    ds_root = Path(ds_root)
    images_root = ds_root / "images"
    labels_root = ds_root / "labels"

    if not images_root.is_dir() or not labels_root.is_dir():
        print(f"⏭️  Skip (missing images/labels): {ds_root}")
        continue

    split_info = detect_split_dirs(ds_root)
    train_dir = split_info["train_img_dir"]
    val_dir   = split_info["val_img_dir"]
    split_mode = split_info["split_mode"]
    train_tag  = split_info.get("train_tag", "train")
    val_tag    = split_info.get("val_tag", "val")

    # --- Calculate image count ---
    if split_mode == "sku_virtual_8_2":
        n_train, n_val = compute_sku_virtual_counts(images_root, seed=SEED, ratio=0.8)
    else:
        n_train = len(list_images(train_dir))
        n_val   = len(list_images(val_dir)) if val_dir else 0

    cases = list_label_cases_for_dataset(ds_root)
    class_names = infer_class_names_from_labels(labels_root)
    nc = len(class_names)

    info = {
        "dataset": ds_root.name,
        "root": str(ds_root),
        "images_root": str(images_root),
        "labels_root": str(labels_root),
        "train_dir": str(train_dir) if train_dir else None,
        "val_dir": str(val_dir) if val_dir else None,
        "n_train": n_train,
        "n_val": n_val,
        "split_mode": split_mode,
        "train_tag": train_tag,
        "val_tag": val_tag,
        "label_cases": [c[0] for c in cases],
        "nc_inferred": nc,
        "class_names_inferred": class_names,
    }
    dataset_summaries.append(info)

    print("\n" + "-" * 80)
    print(f"[Dataset] {ds_root.name}")
    print(f" - root        : {ds_root}")
    print(f" - split_mode  : {split_mode}")
    print(f" - train_dir   : {train_dir if train_dir else '(missing)'} | tag={train_tag} | n_train={n_train}")
    print(f" - val_dir     : {val_dir if val_dir else '(missing)'} | tag={val_tag} | n_val={n_val}")

    # If test structure exists, just report existence (summary counts focus on train/val)
    test_dir = split_info.get("test_img_dir", None)
    if test_dir and test_dir.is_dir():
        n_test = len(list_images(test_dir))
        print(f" - test_dir    : {test_dir} | n_test={n_test}")

    print(f" - label_cases : {[c[0] for c in cases] if cases else '(none)'}")
    print(f" - inferred classes (multiclass-based): nc={nc}, names={class_names}")
    print("-" * 80)

print("\n✅ Cell 1 done.")
print(f"   -> dataset_summaries length = {len(dataset_summaries)}")
print("   -> roots variable is ready for Cell 2.")

[DISCOVERY] Found 13 unique dataset roots under: /home/ISW/project/datasets

--------------------------------------------------------------------------------
[Dataset] SKU-110K
 - root        : /home/ISW/project/datasets/SKU-110K
 - split_mode  : sku_virtual_8_2
 - train_dir   : /home/ISW/project/datasets/SKU-110K/images | tag=virtual_8_2 | n_train=9394
 - val_dir     : /home/ISW/project/datasets/SKU-110K/images | tag=virtual_8_2 | n_val=2349
 - label_cases : ['original', 'scale_0.6', 'scale_0.7', 'scale_0.8', 'scale_0.9', 'scale_1.1', 'scale_1.2', 'scale_1.3', 'scale_1.4', 'side_1', 'side_3', 'side_5', 'side_7', 'side_9']
 - inferred classes (multiclass-based): nc=1, names=['class_0']
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
[Dataset] kitti
 - root        : /home/ISW/project/datasets/kitti
 - split_mode  : explicit
 - train_dir   : /home/ISW/project/datasets/kitti/

# YOLO model train

In [7]:
# ==========================================
# Cell 2) Train & Validate — FINAL (NO VIEW / NO HARDLINK)
#   ✅ images: Use original dataset path as is
#   ✅ labels: ds_root/labels  to symlink "case <-> clean" switching
#   ✅ out_csv: Save eval results based on clean(labels)
#   ⚠️ Concurrent execution (same ds) prohibited: because labels symlink changes
# ==========================================

from __future__ import annotations

import os, json, shutil, random, csv, sys, logging, inspect
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from contextlib import contextmanager

try:
    import cv2
    from PIL import Image
    from tqdm import tqdm
except ImportError:
    print("⚠️ Required libraries are missing. Please install: pip install opencv-python pillow tqdm")
    sys.exit(1)

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

import torch
from ultralytics import YOLO
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# -------------------------------------------------------------------------
# ✅ FIXED grids
# -------------------------------------------------------------------------
UNIFORM_SCALING_FACTORS  = [0.6, 0.8, 1.2, 1.4]
JITTER_PATTERNS = [1, 5, 9]

TRAIN_USE_ORIGINAL    = False
TRAIN_USE_UNIFORM_SCALING_NOISE = True
TRAIN_USE_BOUNDARY_JITTER_NOISE  = True

# Whether to run val inside train (optional)
TRAIN_VAL_DURING_TRAIN = False

NUM_WORKERS = min(8, os.cpu_count() or 4)

IMG_SIZE = 640
EPOCHS   = 10
BATCH    = 32
DEVICE   = "0"
SEED     = 42

OUT_ROOT = Path("/home/ISW/project/object_detect_refine_sam")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

CORRUPT_BACKUP_DIR = Path("/home/ISW/project/_corrupt_files_backup")
CORRUPT_BACKUP_DIR.mkdir(parents=True, exist_ok=True)

CLASS_MODES = ["multiclass"]  # if needed ["multiclass","object_only"]

TARGET_DATASETS: Optional[List[str]] = [
    "kitti",
    "homeobjects-3K",
    "african-wildlife",
    "construction-ppe",
    "brain-tumor",
    "BCCD",
    "signature",
    "medical-pills",
    "VOC",
]
TARGET_DATASETS_LOWER = (
    {name.strip().lower() for name in TARGET_DATASETS}
    if TARGET_DATASETS is not None
    else None
)

REFINE_LABELS_SUBDIR = Path("refine(sam)")

MOVE_LABELS_WITH_CORRUPT_IMAGES = False

DISABLE_ULTRA_CACHE = True
PURGE_RUN_DIR_BEFORE_TRAIN = True

@contextmanager
def suppress_output(enabled: bool = True):
    if not enabled:
        yield
        return
    devnull = open(os.devnull, "w")
    old_out, old_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = devnull, devnull
        yield
    finally:
        sys.stdout, sys.stderr = old_out, old_err
        devnull.close()

# -------------------------------------------------------------------------
# Model specs
# -------------------------------------------------------------------------
YOLOV8N_CKPT_CANDIDATES  = ["yolov8n.pt"]
YOLO11N_CKPT_CANDIDATES  = ["yolo11n.pt", "yolov11n.pt"]

MODEL_SPECS = [
    ("yolov8n", YOLOV8N_CKPT_CANDIDATES),
    ("yolo11n", YOLO11N_CKPT_CANDIDATES),
]

# -------------------------------------------------------------------------
# Utils
# -------------------------------------------------------------------------
_IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def list_images(dir_path: Optional[Path]) -> List[Path]:
    if dir_path is None or not Path(dir_path).exists():
        return []
    dir_path = Path(dir_path)
    imgs = []
    for p in dir_path.rglob("*"):
        if p.is_file() and p.suffix.lower() in _IMG_EXTS:
            imgs.append(p)
    return sorted(imgs)

def _remove_path(p: Path):
    try:
        if (not p.exists()) and (not p.is_symlink()):
            return
        if p.is_symlink() or p.is_file():
            p.unlink(missing_ok=True)
            return
        if p.is_dir():
            shutil.rmtree(p, ignore_errors=True)
            return
    except Exception:
        pass

def safe_rmtree(p: Path):
    _remove_path(p)

def infer_class_names_from_labels(label_root: Path, max_files: int = 2000) -> List[str]:
    if label_root is None or not label_root.exists():
        return ["class_0"]
    txts = list(label_root.rglob("*.txt"))
    if not txts:
        return ["class_0"]

    txts = txts[:max_files]
    cls_ids = set()
    for t in txts:
        try:
            with open(t, "r", encoding="utf-8") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    cid = int(float(parts[0]))
                    cls_ids.add(cid)
        except Exception:
            continue

    if not cls_ids:
        return ["class_0"]
    max_id = max(cls_ids)
    return [f"class_{i}" for i in range(max_id + 1)]

def choose_model(ckpt_candidates: List[str]) -> YOLO:
    last_err = None
    for ckpt in ckpt_candidates:
        try:
            return YOLO(ckpt)
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Failed to load model weights: {ckpt_candidates}") from last_err

def extract_metrics_dict(val_result) -> Dict:
    if val_result is None:
        return {}
    if isinstance(val_result, dict):
        return dict(val_result)
    if hasattr(val_result, "results_dict") and isinstance(val_result.results_dict, dict):
        return dict(val_result.results_dict)
    try:
        d = dict(val_result.__dict__)
        d.pop("plots", None)
        d.pop("speed", None)
        return d
    except Exception:
        return {}

def _supports_kwarg(fn, name: str) -> bool:
    try:
        sig = inspect.signature(fn)
        return name in sig.parameters
    except Exception:
        return False

def pick_eval_weight(run_dir: Path, train_val: bool) -> Tuple[Optional[Path], str]:
    wdir = run_dir / "weights"
    best_pt = wdir / "best.pt"
    last_pt = wdir / "last.pt"
    if train_val and best_pt.exists():
        return best_pt, "best.pt"
    if last_pt.exists():
        return last_pt, "last.pt"
    if best_pt.exists():
        return best_pt, "best.pt"
    return None, "missing"

def _img2label_path(img_path: Path) -> Path:
    s = str(img_path)
    s = s.replace(f"{os.sep}images{os.sep}", f"{os.sep}labels{os.sep}")
    return Path(s).with_suffix(".txt")

# -------------------------------------------------------------------------
# Corrupt cleaner
# -------------------------------------------------------------------------
def scan_and_clean_images(images_dir: Path, ds_root: Path):
    if not images_dir.exists():
        return
    images = list_images(images_dir)
    if not images:
        return

    print(f"   🔍 Scanning integrity of {len(images)} images in {images_dir.name}...")
    corrupt_count = 0
    for img_path in tqdm(images, desc="Checking", leave=False):
        is_corrupt = False
        try:
            with Image.open(img_path) as im:
                im.verify()
        except Exception:
            is_corrupt = True

        if not is_corrupt:
            try:
                img = cv2.imread(str(img_path))
                if img is None:
                    is_corrupt = True
                else:
                    _ = img.shape
            except Exception:
                is_corrupt = True

        if is_corrupt:
            corrupt_count += 1
            dest = CORRUPT_BACKUP_DIR / ds_root.name / img_path.relative_to(images_dir)
            dest.parent.mkdir(parents=True, exist_ok=True)
            try:
                shutil.move(str(img_path), str(dest))
            except Exception:
                pass

            if MOVE_LABELS_WITH_CORRUPT_IMAGES:
                label_guess = ds_root / "labels" / img_path.relative_to(images_dir).with_suffix(".txt")
                try:
                    if label_guess.exists():
                        shutil.move(str(label_guess), str(dest.with_suffix(".txt")))
                except Exception:
                    pass

    if corrupt_count > 0:
        print(f"   ⚠️  Moved {corrupt_count} corrupt images to {CORRUPT_BACKUP_DIR}")
    else:
        print(f"   ✅  No corrupt images found.")

# -------------------------------------------------------------------------
# Split/Case helpers
# -------------------------------------------------------------------------
def normalize_name(name: str) -> str:
    return name.strip().lower().replace("_", "-").replace(" ", "-")

_ds_map: Dict[str, Dict] = {}
try:
    for info in dataset_summaries:
        key = normalize_name(info.get("dataset", ""))
        if key:
            _ds_map[key] = info
except Exception:
    _ds_map = {}

def get_split_info(ds_root: Path) -> Dict[str, Optional[Path]]:
    key = normalize_name(ds_root.name)
    if key in _ds_map:
        info = _ds_map[key]
        return dict(
            train_img_dir=Path(info["train_dir"]) if info.get("train_dir") else None,
            val_img_dir=Path(info["val_dir"]) if info.get("val_dir") else None,
            split_mode=info.get("split_mode", "fallback"),
            train_tag=info.get("train_tag", "train"),
            val_tag=info.get("val_tag", "val"),
        )

    images_root = ds_root / "images"
    tr = images_root / "train" if (images_root / "train").is_dir() else images_root
    va = images_root / "val" if (images_root / "val").is_dir() else (images_root / "valid" if (images_root / "valid").is_dir() else None)
    return dict(
        train_img_dir=tr,
        val_img_dir=va,
        split_mode="fallback",
        train_tag=tr.name if tr else "unknown",
        val_tag=va.name if va else "missing",
    )

def get_refine_labels_root(ds_root: Path) -> Path:
    return ds_root / REFINE_LABELS_SUBDIR

def list_label_cases_for_dataset(ds_root: Path) -> List[Tuple[str, str]]:
    refine_root = get_refine_labels_root(ds_root)
    cases: List[Tuple[str, str]] = []

    if TRAIN_USE_ORIGINAL:
        if (ds_root / "labels").is_dir() or (ds_root / "labels").is_symlink():
            cases.append(("original", "labels"))

    if TRAIN_USE_UNIFORM_SCALING_NOISE and refine_root.is_dir():
        for s in UNIFORM_SCALING_FACTORS:
            tag = f"scale_{s}"
            dirname = f"labels_uniform_scaling_{s}"
            if (refine_root / dirname).is_dir():
                cases.append((tag, dirname))

    if TRAIN_USE_BOUNDARY_JITTER_NOISE and refine_root.is_dir():
        for k in JITTER_PATTERNS:
            tag = f"side_{k}"
            dirname = f"labels_boundary_jitter_{k}"
            if (refine_root / dirname).is_dir():
                cases.append((tag, dirname))

    return cases

# -------------------------------------------------------------------------
# ✅ Core: Make ds_root/labels switchable (case <-> clean), yaml uses ds_root/images as is
# -------------------------------------------------------------------------
def prepare_switchable_labels(ds_root: Path) -> Tuple[Path, Path]:
    """
    Returns: (labels_link_path, clean_labels_root_real)
    - If ds_root/labels is directory: rename labels -> labels__CLEAN, then create labels symlink
    - If ds_root/labels is already symlink: consider its target as clean
    """
    labels_link = ds_root / "labels"
    clean_root  = None

    if labels_link.is_symlink():
        # Consider already switchable state
        try:
            clean_root = Path(os.readlink(labels_link))
            if not clean_root.is_absolute():
                clean_root = (labels_link.parent / clean_root).resolve()
        except Exception:
            clean_root = labels_link.resolve()
        return labels_link, clean_root

    # labels "real folder"is case: rename  then symlink convert switching possible make
    if labels_link.is_dir():
        clean_root = ds_root / "labels__CLEAN"
        if clean_root.exists():
            # If already exists, use labels__CLEAN as clean and set labels as symlink
            pass
        else:
            labels_link.rename(clean_root)

        # Create labels -> clean_root symlink
        _remove_path(labels_link)
        os.symlink(str(clean_root), str(labels_link))
        return labels_link, clean_root

    raise RuntimeError(f"clean labels folder not found: {labels_link}")

def switch_labels(ds_root: Path, target_labels_root: Path):
    """
    Switch ds_root/labels symlink to target_labels_root
    """
    labels_link = ds_root / "labels"
    _remove_path(labels_link)
    os.symlink(str(target_labels_root), str(labels_link))

def write_data_yaml_abs(yaml_path: Path, train_img_dir: Path, val_img_dir: Path, names: List[str]):
    nc = len(names)
    yaml_path.parent.mkdir(parents=True, exist_ok=True)
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write(
            f"train: {str(train_img_dir)}\n"
            f"val: {str(val_img_dir)}\n"
            f"nc: {nc}\n"
            f"names: {names}\n"
        )

def debug_show_one_mapping_from_real(ds_root: Path, val_img_dir: Path, tag: str):
    imgs = list_images(val_img_dir)
    if not imgs:
        print(f"   🔎 [{tag}] no val images.")
        return
    img0 = imgs[0]
    lbl0 = _img2label_path(img0)
    link = ds_root / "labels"
    try:
        link_target = os.readlink(link) if link.is_symlink() else "(not a symlink)"
    except Exception as e:
        link_target = f"(readlink failed: {e})"
    try:
        lbl0_real = lbl0.resolve()
    except Exception:
        lbl0_real = lbl0

    print(f"   🔎 [{tag}] ds_root/labels link -> {link_target}")
    print(f"   🔎 [{tag}] sample img         : {img0}")
    print(f"   🔎 [{tag}] mapped lbl         : {lbl0} | exists={lbl0.exists()}")
    print(f"   🔎 [{tag}] mapped lbl real    : {lbl0_real}")

# -------------------------------------------------------------------------
# Train/Eval wrappers
# -------------------------------------------------------------------------
def train_with_auto_oom(
    model: YOLO,
    data_yaml: Path,
    project_dir: Path,
    name_dir: str,
    model_tag: str,
    single_cls: bool,
    train_val: bool,
):
    candidates = [(BATCH, IMG_SIZE)]
    last_err = None
    for b, sz in candidates:
        try:
            kwargs = dict(
                data=str(data_yaml),
                epochs=EPOCHS,
                imgsz=sz,
                batch=b,
                device=DEVICE,
                project=str(project_dir),
                name=name_dir,
                exist_ok=True,
                verbose=False,
                workers=NUM_WORKERS,
                amp=True,
                single_cls=single_cls,
            )
            if _supports_kwarg(model.train, "val"):
                kwargs["val"] = train_val
            if DISABLE_ULTRA_CACHE and _supports_kwarg(model.train, "cache"):
                kwargs["cache"] = False

            with suppress_output(True):
                train_res = model.train(**kwargs)
            return True, b, sz, train_res, None

        except RuntimeError as e:
            msg = str(e).lower()
            last_err = e
            if "out of memory" in msg or "cuda out of memory" in msg:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    torch.cuda.ipc_collect()
                continue
            break
        except Exception as e:
            last_err = e
            break

    return False, None, None, None, last_err

def val_with_yaml(model: YOLO, data_yaml: Path, imgsz: int, single_cls: bool) -> Dict:
    kwargs = dict(
        data=str(data_yaml),
        imgsz=imgsz,
        device=DEVICE,
        split="val",
        verbose=False,
        workers=NUM_WORKERS,
        single_cls=single_cls,
    )
    if DISABLE_ULTRA_CACHE and _supports_kwarg(model.val, "cache"):
        kwargs["cache"] = False
    with suppress_output(True):
        res = model.val(**kwargs)
    return extract_metrics_dict(res)

# -------------------------------------------------------------------------
# MAIN
# -------------------------------------------------------------------------
set_seed(SEED)

print("=" * 80)
print("[TRAIN/EVAL] Start (DIRECT dataset images + labels symlink switching ONLY)")
print(f" - OUT_ROOT               : {OUT_ROOT}")
print(f" - NUM_WORKERS            : {NUM_WORKERS}")
print(f" - CLASS_MODES            : {CLASS_MODES}")
print(f" - REFINE_LABELS_SUBDIR   : {REFINE_LABELS_SUBDIR}")
print(f" - TRAIN_USE_ORIGINAL     : {TRAIN_USE_ORIGINAL}")
print(f" - TRAIN_USE_UNIFORM_SCALING_NOISE  : {TRAIN_USE_UNIFORM_SCALING_NOISE}")
print(f" - TRAIN_USE_BOUNDARY_JITTER_NOISE   : {TRAIN_USE_BOUNDARY_JITTER_NOISE}")
print(f" - TRAIN_VAL_DURING_TRAIN : {TRAIN_VAL_DURING_TRAIN}")
print(f" - DISABLE_ULTRA_CACHE    : {DISABLE_ULTRA_CACHE}")
print(f" - PURGE_RUN_DIR_BEFORE_TRAIN: {PURGE_RUN_DIR_BEFORE_TRAIN}")
if TARGET_DATASETS_LOWER is None:
    print(f" - TARGET_DATASETS        : ALL")
else:
    print(f" - TARGET_DATASETS        : {sorted(TARGET_DATASETS_LOWER)}")
print("=" * 80)

summary_rows: List[Dict] = []

try:
    _ = roots
except NameError:
    raise RuntimeError("Please run Cell 1 first to prepare the roots variable.")

for ds_root in roots:
    ds_root = Path(ds_root)

    if TARGET_DATASETS_LOWER is not None:
        if ds_root.name.strip().lower() not in TARGET_DATASETS_LOWER:
            print(f"⏭️  Skip (not in TARGET_DATASETS): {ds_root.name}")
            continue

    images_root = ds_root / "images"
    if not images_root.is_dir():
        print(f"⏭️  Skip (missing images): {ds_root}")
        continue

    print(f"\n[Integrity Check] {ds_root.name}")
    scan_and_clean_images(images_root, ds_root)

    cases = list_label_cases_for_dataset(ds_root)
    if not cases:
        print(f"⏭️  Skip (no target label cases found): {ds_root.name}")
        print(f"   - expected refine root: {get_refine_labels_root(ds_root)}")
        continue

    sp = get_split_info(ds_root)
    train_img_dir = sp["train_img_dir"]
    val_img_dir   = sp["val_img_dir"]
    split_mode    = sp.get("split_mode", "fallback")

    if train_img_dir is None or not Path(train_img_dir).is_dir():
        print(f"⏭️  Skip (No train images dir resolved): {ds_root.name}")
        continue
    if val_img_dir is None or not Path(val_img_dir).is_dir():
        print(f"⏭️  Skip (No val images dir resolved): {ds_root.name}")
        continue

    # ✅ Prepare labels for switching (one-time rename + symlink)
    labels_link, clean_labels_root = prepare_switchable_labels(ds_root)

    # ✅ names/nc based on clean
    names_clean = infer_class_names_from_labels(clean_labels_root)
    if not names_clean:
        names_clean = ["class_0"]

    # ✅ dataset yaml (images use original path absolute)
    yaml_dir = OUT_ROOT / "_yamls" / ds_root.name
    train_yaml = yaml_dir / "data_train.yaml"
    eval_yaml  = yaml_dir / "data_eval_clean.yaml"
    write_data_yaml_abs(train_yaml, Path(train_img_dir), Path(val_img_dir), names_clean)
    write_data_yaml_abs(eval_yaml,  Path(train_img_dir), Path(val_img_dir), names_clean)

    print("\n" + "-" * 80)
    print(f"[Dataset] {ds_root.name}")
    print(f" - ds_root            : {ds_root}")
    print(f" - train_img_dir      : {train_img_dir}")
    print(f" - val_img_dir        : {val_img_dir}")
    print(f" - labels_link        : {labels_link}")
    print(f" - clean_labels_root  : {clean_labels_root}")
    print(f" - split_mode         : {split_mode}")
    print(f" - Cases              : {[c[0] for c in cases]}")
    print("-" * 80)

    # dataset level: restore to clean at the end no matter what
    try:
        for case_tag, labels_dirname in cases:
            refine_root = get_refine_labels_root(ds_root)
            case_label_root = (ds_root / "labels") if labels_dirname == "labels" else (refine_root / labels_dirname)
            if labels_dirname == "labels":
                # For original, use clean_root to train with clean root
                case_label_root = clean_labels_root

            if not case_label_root.is_dir():
                print(f"  ⏭️  Skip (case label root missing): {case_label_root}")
                continue

            # ✅ (Core) Switch labels to case just before training
            switch_labels(ds_root, case_label_root)
            debug_show_one_mapping_from_real(ds_root, Path(val_img_dir), tag=f"TRAIN(case={case_tag})")

            for class_mode in CLASS_MODES:
                single_cls = (class_mode == "object_only")

                for model_tag, ckpt_candidates in MODEL_SPECS:
                    print(f"\n  [Train] case={case_tag} | class_mode={class_mode} | model={model_tag} (single_cls={single_cls})")

                    try:
                        model = choose_model(ckpt_candidates)
                    except Exception as e:
                        print(f"    ❌ Model load failed: {ckpt_candidates} | err={e}")
                        continue

                    project_dir = OUT_ROOT / ds_root.name
                    name_dir = f"{model_tag}__{case_tag}__{class_mode}__DIRECT"
                    run_dir = project_dir / name_dir
                    project_dir.mkdir(parents=True, exist_ok=True)

                    if PURGE_RUN_DIR_BEFORE_TRAIN:
                        safe_rmtree(run_dir)

                    # Force back to case in case it changed midway
                    switch_labels(ds_root, case_label_root)

                    ok, used_b, used_sz, train_res, err = train_with_auto_oom(
                        model=model,
                        data_yaml=train_yaml,
                        project_dir=project_dir,
                        name_dir=name_dir,
                        model_tag=model_tag,
                        single_cls=single_cls,
                        train_val=TRAIN_VAL_DURING_TRAIN,
                    )
                    if not ok:
                        print(f"    ❌ Train failed: {err}")
                        try:
                            del model
                        except Exception:
                            pass
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                            torch.cuda.ipc_collect()
                        continue

                    eval_w, eval_w_name = pick_eval_weight(run_dir, train_val=TRAIN_VAL_DURING_TRAIN)

                    # ✅ clean eval
                    metrics_clean = {}
                    if eval_w is None:
                        print(f"    ⚠️  Weights missing: {run_dir/'weights'} -> skip clean eval")
                    else:
                        switch_labels(ds_root, clean_labels_root)
                        debug_show_one_mapping_from_real(ds_root, Path(val_img_dir), tag=f"EVAL_CLEAN(case={case_tag})")
                        try:
                            eval_model = YOLO(str(eval_w))
                            metrics_clean = val_with_yaml(
                                model=eval_model,
                                data_yaml=eval_yaml,
                                imgsz=used_sz if used_sz else IMG_SIZE,
                                single_cls=single_cls,
                            )
                        except Exception as e:
                            print(f"    ❌ Clean eval failed: {e}")
                            metrics_clean = {}

                        # Restore to case for next training
                        switch_labels(ds_root, case_label_root)

                    # ✅ Save clean eval to run_dir
                    metrics_out = run_dir / "metrics_eval_clean.json"
                    try:
                        metrics_out.parent.mkdir(parents=True, exist_ok=True)
                        with open(metrics_out, "w", encoding="utf-8") as f:
                            json.dump(
                                dict(
                                    dataset=ds_root.name,
                                    ds_root=str(ds_root),
                                    case_tag=case_tag,
                                    labels_dirname=labels_dirname,
                                    train_labels_root_used=str(case_label_root),
                                    clean_labels_root=str(clean_labels_root),
                                    class_mode=class_mode,
                                    single_cls=single_cls,
                                    model_tag=model_tag,
                                    ckpt_candidates=ckpt_candidates,
                                    train_yaml=str(train_yaml),
                                    eval_yaml_clean=str(eval_yaml),
                                    effective_batch=used_b,
                                    effective_imgsz=used_sz,
                                    train_val_during_train=TRAIN_VAL_DURING_TRAIN,
                                    run_dir=str(run_dir),
                                    eval_weight_pick=eval_w_name,
                                    eval_weight_path=str(eval_w) if eval_w is not None else None,
                                    metrics_clean=metrics_clean,
                                ),
                                f,
                                ensure_ascii=False,
                                indent=2,
                            )
                        print(f"    ✅ Saved clean metrics: {metrics_out}")
                    except Exception:
                        pass

                    # ✅ summary row
                    row = dict(
                        dataset=ds_root.name,
                        model=model_tag,
                        case=case_tag,
                        labels_dir=labels_dirname,
                        class_mode=class_mode,
                        single_cls=single_cls,
                        split_mode=split_mode,
                        effective_batch=used_b,
                        effective_imgsz=used_sz,
                        train_val_during_train=TRAIN_VAL_DURING_TRAIN,
                        eval_weight_pick=eval_w_name,
                        eval_weight_path=str(eval_w) if eval_w is not None else None,
                        clean_eval_yaml=str(eval_yaml),
                    )
                    for k in ["metrics/mAP50(B)", "metrics/mAP50-95(B)", "metrics/precision(B)", "metrics/recall(B)"]:
                        if k in metrics_clean:
                            row[k] = metrics_clean[k]
                    summary_rows.append(row)

                    try:
                        del model
                    except Exception:
                        pass
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                        torch.cuda.ipc_collect()

    finally:
        # ✅ Always restore to clean when dataset processing ends
        try:
            switch_labels(ds_root, clean_labels_root)
        except Exception:
            pass

# -------------------------------------------------------------------------
# Save summary CSV (based on clean eval)
# -------------------------------------------------------------------------
out_csv = OUT_ROOT / "object_detection_proposed(100).csv"
try:
    cols = set()
    for r in summary_rows:
        cols.update(r.keys())

    base_cols = [
        "dataset", "model", "case", "labels_dir",
        "class_mode", "single_cls",
        "split_mode",
        "effective_batch", "effective_imgsz",
        "train_val_during_train",
        "eval_weight_pick", "eval_weight_path",
        "metrics/mAP50(B)", "metrics/mAP50-95(B)", "metrics/precision(B)", "metrics/recall(B)",
        "clean_eval_yaml",
    ]
    extra_cols = sorted([c for c in cols if c not in set(base_cols)])
    cols = base_cols + extra_cols

    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for r in summary_rows:
            w.writerow(r)

    print("\n" + "=" * 80)
    print(f"✅ Saved summary CSV (eval=clean labels): {out_csv}")
    print(f"✅ Total runs: {len(summary_rows)}")
    print("=" * 80)

except Exception as e:
    print(f"⚠️  Summary CSV save failed: {e}")

print("\n✅ Cell 2 (DIRECT) done.")


[TRAIN/EVAL] Start (DIRECT dataset images + labels symlink switching ONLY)
 - OUT_ROOT               : /home/ISW/project/object_detect_refine_sam
 - NUM_WORKERS            : 8
 - CLASS_MODES            : ['multiclass']
 - REFINE_LABELS_SUBDIR   : refine(sam)
 - TRAIN_USE_ORIGINAL     : False
 - TRAIN_USE_UNIFORM_SCALING_NOISE  : True
 - TRAIN_USE_BOUNDARY_JITTER_NOISE   : True
 - TRAIN_VAL_DURING_TRAIN : False
 - DISABLE_ULTRA_CACHE    : True
 - PURGE_RUN_DIR_BEFORE_TRAIN: True
 - TARGET_DATASETS        : ['african-wildlife', 'bccd', 'brain-tumor', 'construction-ppe', 'homeobjects-3k', 'kitti', 'medical-pills', 'signature', 'voc']
⏭️  Skip (not in TARGET_DATASETS): SKU-110K

[Integrity Check] kitti
   🔍 Scanning integrity of 7481 images in images...


   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] kitti
 - ds_root            : /home/ISW/project/datasets/kitti
 - train_img_dir      : /home/ISW/project/datasets/kitti/images/train
 - val_img_dir        : /home/ISW/project/datasets/kitti/images/val
 - labels_link        : /home/ISW/project/datasets/kitti/labels
 - clean_labels_root  : /home/ISW/project/datasets/kitti/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/kitti/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/kitti/images/val/000000.png
   🔎 [TRAIN(case=scale_0.6)] mapped lbl         : /home/ISW/projec

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] homeobjects-3K
 - ds_root            : /home/ISW/project/datasets/homeobjects-3K
 - train_img_dir      : /home/ISW/project/datasets/homeobjects-3K/images/train
 - val_img_dir        : /home/ISW/project/datasets/homeobjects-3K/images/val
 - labels_link        : /home/ISW/project/datasets/homeobjects-3K/labels
 - clean_labels_root  : /home/ISW/project/datasets/homeobjects-3K/labels__CLEAN
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/homeobjects-3K/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/homeobjects-3K/images/val/living_room_10.jpg
   🔎 [TRAIN

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] african-wildlife
 - ds_root            : /home/ISW/project/datasets/african-wildlife
 - train_img_dir      : /home/ISW/project/datasets/african-wildlife/images/train
 - val_img_dir        : /home/ISW/project/datasets/african-wildlife/images/val
 - labels_link        : /home/ISW/project/datasets/african-wildlife/labels
 - clean_labels_root  : /home/ISW/project/datasets/african-wildlife/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/african-wildlife/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/african-wildlife/

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] construction-ppe
 - ds_root            : /home/ISW/project/datasets/construction-ppe
 - train_img_dir      : /home/ISW/project/datasets/construction-ppe/images/train
 - val_img_dir        : /home/ISW/project/datasets/construction-ppe/images/val
 - labels_link        : /home/ISW/project/datasets/construction-ppe/labels
 - clean_labels_root  : /home/ISW/project/datasets/construction-ppe/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/construction-ppe/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/construction-ppe/

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] brain-tumor
 - ds_root            : /home/ISW/project/datasets/brain-tumor
 - train_img_dir      : /home/ISW/project/datasets/brain-tumor/images/train
 - val_img_dir        : /home/ISW/project/datasets/brain-tumor/images/val
 - labels_link        : /home/ISW/project/datasets/brain-tumor/labels
 - clean_labels_root  : /home/ISW/project/datasets/brain-tumor/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/brain-tumor/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/brain-tumor/images/val/val_1 (1).jpg
   🔎 [TRAIN(cas

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] BCCD
 - ds_root            : /home/ISW/project/datasets/BCCD
 - train_img_dir      : /home/ISW/project/datasets/BCCD/images/train
 - val_img_dir        : /home/ISW/project/datasets/BCCD/images/val
 - labels_link        : /home/ISW/project/datasets/BCCD/labels
 - clean_labels_root  : /home/ISW/project/datasets/BCCD/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/BCCD/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/BCCD/images/val/BloodImage_00012.jpg
   🔎 [TRAIN(case=scale_0.6)] mapped lbl         : /home/ISW/proj

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] signature
 - ds_root            : /home/ISW/project/datasets/signature
 - train_img_dir      : /home/ISW/project/datasets/signature/images/train
 - val_img_dir        : /home/ISW/project/datasets/signature/images/val
 - labels_link        : /home/ISW/project/datasets/signature/labels
 - clean_labels_root  : /home/ISW/project/datasets/signature/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/signature/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/signature/images/val/Frame_108.jpg
   🔎 [TRAIN(case=scale_0.6)] ma

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] medical-pills
 - ds_root            : /home/ISW/project/datasets/medical-pills
 - train_img_dir      : /home/ISW/project/datasets/medical-pills/images/train
 - val_img_dir        : /home/ISW/project/datasets/medical-pills/images/val
 - labels_link        : /home/ISW/project/datasets/medical-pills/labels
 - clean_labels_root  : /home/ISW/project/datasets/medical-pills/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/medical-pills/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/medical-pills/images/val/Frame_144.jpg

   ✅  No corrupt images found.

--------------------------------------------------------------------------------
[Dataset] VOC
 - ds_root            : /home/ISW/project/datasets/VOC
 - train_img_dir      : /home/ISW/project/datasets/VOC/images/train2012
 - val_img_dir        : /home/ISW/project/datasets/VOC/images/val2012
 - labels_link        : /home/ISW/project/datasets/VOC/labels
 - clean_labels_root  : /home/ISW/project/datasets/VOC/refine(100)/labels_uniform_scaling_0.6
 - split_mode         : explicit
 - Cases              : ['scale_0.6', 'scale_0.8', 'scale_1.2', 'scale_1.4', 'side_1', 'side_5', 'side_9']
--------------------------------------------------------------------------------
   🔎 [TRAIN(case=scale_0.6)] ds_root/labels link -> /home/ISW/project/datasets/VOC/refine(sam)/labels_uniform_scaling_0.6
   🔎 [TRAIN(case=scale_0.6)] sample img         : /home/ISW/project/datasets/VOC/images/val2012/2008_000002.jpg
   🔎 [TRAIN(case=scale_0.6)] mapped lbl         : /home/ISW/proje

# Load trained object detection model & evaluate with original labels

In [8]:
# ==========================================
# Cell 3) Evaluate trained weights on CLEAN labels (ds_root/labels)
#   - Load saved weights from Cell 2 OUT_ROOT
#   - For each dataset / (scale,side) case / model / class_mode:
#       evaluate on CLEAN GT = /datasets/{ds}/labels (val split)
#   - NO COPY: runtime view = directory symlinks + per-run data.yaml
#   - Output:
#       (1) run_dir/metrics_on_clean.json
#       (2) OUT_ROOT/summary_eval_on_clean_labels.csv
# ==========================================

from __future__ import annotations

import os, json, csv, shutil, sys, logging, inspect
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple
from contextlib import contextmanager

import torch
from ultralytics import YOLO
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# -------------------------
# MUST MATCH Cell 2 SETTINGS (or override)
# -------------------------
OUT_ROOT = Path("/home/ISW/project/object_detect_refine_sam").resolve()
DEVICE   = "0"
NUM_WORKERS = min(8, os.cpu_count() or 4)
IMG_SIZE = 640

# same grids as Cell 2
# UNIFORM_SCALING_FACTORS  = [0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4]
# JITTER_PATTERNS = [1, 3, 5, 7, 9]

# evaluate only these cases (recommended)
EVAL_ONLY_NOISE_CASES = True   # True: only scale_* and side_* (limited to above factors)
CLEANUP_RUNTIME_VROOT  = True
SILENCE_ULTRA_OUTPUT   = True

# runtime view base (separate from training view)
RUNTIME_EVAL_VROOT_BASE = Path("/home/ISW/project/_runtime_eval_views")
RUNTIME_EVAL_VROOT_BASE.mkdir(parents=True, exist_ok=True)

# -------------------------
# Helpers (safe output, symlink, split detection)
# -------------------------
@contextmanager
def suppress_output(enabled: bool = True):
    if not enabled:
        yield
        return
    devnull = open(os.devnull, "w")
    old_out, old_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = devnull, devnull
        yield
    finally:
        sys.stdout, sys.stderr = old_out, old_err
        devnull.close()

def safe_rmtree(p: Path):
    try:
        if p.exists() or p.is_symlink():
            shutil.rmtree(p, ignore_errors=True)
    except Exception:
        pass

def safe_symlink(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists() or dst.is_symlink():
        return
    os.symlink(str(src), str(dst))

def normalize_name(name: str) -> str:
    return name.strip().lower().replace("_", "-").replace(" ", "-")

# If Cell 1 has dataset_summaries, use it. Otherwise fallback to common layouts.
_ds_map: Dict[str, Dict[str, Any]] = {}
try:
    for info in dataset_summaries:
        key = normalize_name(info.get("dataset", ""))
        if key:
            _ds_map[key] = info
except Exception:
    _ds_map = {}

def get_split_info(ds_root: Path) -> Dict[str, Optional[Path]]:
    key = normalize_name(ds_root.name)
    if key in _ds_map:
        info = _ds_map[key]
        return dict(
            train_img_dir=Path(info["train_dir"]) if info.get("train_dir") else None,
            val_img_dir=Path(info["val_dir"]) if info.get("val_dir") else None,
            split_mode=info.get("split_mode", "from_dataset_summaries"),
            train_tag=info.get("train_tag", "train"),
            val_tag=info.get("val_tag", "val"),
        )

    images_root = ds_root / "images"
    tr = images_root / "train" if (images_root / "train").is_dir() else images_root
    va = None
    if (images_root / "val").is_dir():
        va = images_root / "val"
    elif (images_root / "valid").is_dir():
        va = images_root / "valid"
    return dict(
        train_img_dir=tr if tr.is_dir() else None,
        val_img_dir=va if (va and va.is_dir()) else None,
        split_mode="fallback",
        train_tag=tr.name if tr else "train",
        val_tag=va.name if va else "val",
    )

def resolve_label_base(label_root: Path, split_tag: str, is_train: bool) -> Path:
    """
    If split folders like train2012/val2012 exist under label_root, use them first,
    otherwise fallback to train/val/valid, finally use label_root itself.
    """
    tagged = label_root / split_tag if split_tag else None
    if tagged and tagged.is_dir():
        return tagged

    if is_train:
        tdir = label_root / "train"
        return tdir if tdir.is_dir() else label_root
    else:
        vdir = label_root / "val"
        if vdir.is_dir():
            return vdir
        vdir2 = label_root / "valid"
        if vdir2.is_dir():
            return vdir2
        return label_root

def infer_class_names_from_labels(label_root: Path, max_files: int = 2000) -> List[str]:
    if label_root is None or not label_root.exists():
        return ["class_0"]
    txts = list(label_root.rglob("*.txt"))
    if not txts:
        return ["class_0"]

    txts = txts[:max_files]
    cls_ids = set()
    for t in txts:
        try:
            with open(t, "r", encoding="utf-8") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    cls_ids.add(int(float(parts[0])))
        except Exception:
            continue

    if not cls_ids:
        return ["class_0"]
    max_id = max(cls_ids)
    return [f"class_{i}" for i in range(max_id + 1)]

def extract_metrics_dict(val_result) -> Dict[str, Any]:
    if val_result is None:
        return {}
    if isinstance(val_result, dict):
        return dict(val_result)
    if hasattr(val_result, "results_dict") and isinstance(val_result.results_dict, dict):
        return dict(val_result.results_dict)
    try:
        d = dict(val_result.__dict__)
        d.pop("plots", None)
        d.pop("speed", None)
        return d
    except Exception:
        return {}

def pick_weight_file(run_dir: Path) -> Tuple[Optional[Path], str]:
    wdir = run_dir / "weights"
    best_pt = wdir / "best.pt"
    last_pt = wdir / "last.pt"
    if best_pt.exists():
        return best_pt, "best.pt"
    if last_pt.exists():
        return last_pt, "last.pt"
    # fallback: any *.pt
    pts = sorted(wdir.glob("*.pt"))
    if pts:
        return pts[-1], pts[-1].name
    return None, "missing"

def _safe_tag(s: str) -> str:
    s = str(s).replace("\\", "/")
    s = s.replace("/", "__").replace(" ", "_")
    return s

def build_clean_runtime_view_and_yaml(ds_root: Path) -> Tuple[Path, Path, str]:
    """
    Build evaluation-only runtime view:
      vroot/images/train -> original train images dir (symlink)
      vroot/images/val   -> original val images dir   (symlink)
      vroot/labels/train -> CLEAN labels train base   (symlink)
      vroot/labels/val   -> CLEAN labels val base     (symlink)
    """
    sp = get_split_info(ds_root)
    train_img_dir = sp["train_img_dir"]
    val_img_dir   = sp["val_img_dir"]
    split_mode    = sp.get("split_mode", "fallback")
    train_tag     = sp.get("train_tag", "train")
    val_tag       = sp.get("val_tag", "val")

    if train_img_dir is None or not Path(train_img_dir).is_dir():
        raise RuntimeError(f"No train images dir resolved: {ds_root}")
    if val_img_dir is None or not Path(val_img_dir).is_dir():
        raise RuntimeError(f"No val images dir resolved: {ds_root}")

    clean_label_root = ds_root / "labels"
    if not clean_label_root.is_dir():
        raise RuntimeError(f"CLEAN label root missing: {clean_label_root}")

    vroot = RUNTIME_EVAL_VROOT_BASE / ds_root.name / "clean_eval_view"
    safe_rmtree(vroot)
    (vroot / "images").mkdir(parents=True, exist_ok=True)
    (vroot / "labels").mkdir(parents=True, exist_ok=True)

    # images dirs
    safe_symlink(Path(train_img_dir), vroot / "images" / "train")
    safe_symlink(Path(val_img_dir),   vroot / "images" / "val")

    # labels dirs (resolve split subfolder if any)
    clean_train_lbl = resolve_label_base(clean_label_root, train_tag, is_train=True)
    clean_val_lbl   = resolve_label_base(clean_label_root, val_tag,   is_train=False)
    if not clean_train_lbl.is_dir():
        raise RuntimeError(f"CLEAN train labels dir missing: {clean_train_lbl}")
    if not clean_val_lbl.is_dir():
        raise RuntimeError(f"CLEAN val labels dir missing: {clean_val_lbl}")

    safe_symlink(clean_train_lbl, vroot / "labels" / "train")
    safe_symlink(clean_val_lbl,   vroot / "labels" / "val")

    # yaml
    names = infer_class_names_from_labels(clean_label_root)
    nc = len(names)

    data_yaml = vroot / "data.yaml"
    with open(data_yaml, "w", encoding="utf-8") as f:
        f.write(
            f"path: {str(vroot)}\n"
            f"train: images/train\n"
            f"val: images/val\n"
            f"nc: {nc}\n"
            f"names: {names}\n"
        )

    return vroot, data_yaml, split_mode

def should_eval_case(case_tag: str) -> bool:
    if not EVAL_ONLY_NOISE_CASES:
        return True
    if case_tag.startswith("scale_"):
        try:
            s = float(case_tag.split("_", 1)[1])
            return s in UNIFORM_SCALING_FACTORS
        except Exception:
            return False
    if case_tag.startswith("side_"):
        try:
            k = int(case_tag.split("_", 1)[1])
            return k in JITTER_PATTERNS
        except Exception:
            return False
    return False

# -------------------------
# 1) Collect runs from OUT_ROOT
# -------------------------
if not OUT_ROOT.exists():
    raise RuntimeError(f"OUT_ROOT not found: {OUT_ROOT}")

# dataset directories under OUT_ROOT (created by Cell2: OUT_ROOT/<dataset>/<run_name>/...)
ds_dirs = [p for p in OUT_ROOT.iterdir() if p.is_dir()]

print("=" * 80)
print("[EVAL on CLEAN] Load Cell2 weights and evaluate on ds_root/labels (CLEAN)")
print(f" - OUT_ROOT               : {OUT_ROOT}")
print(f" - RUNTIME_EVAL_VROOT_BASE : {RUNTIME_EVAL_VROOT_BASE}")
print(f" - DEVICE                 : {DEVICE}")
print(f" - NUM_WORKERS            : {NUM_WORKERS}")
print(f" - IMG_SIZE(default)      : {IMG_SIZE}")
print(f" - EVAL_ONLY_NOISE_CASES  : {EVAL_ONLY_NOISE_CASES} (scale/side only)")
print("=" * 80)

# We need dataset roots (same as Cell2 loop used `roots`)
try:
    _ = roots
except NameError:
    raise RuntimeError("Please run Cell 1 first to prepare the roots variable. (datasets roots list)")

# Map dataset name -> ds_root path (from roots)
roots_map: Dict[str, Path] = {}
for r in roots:
    rr = Path(r)
    roots_map[rr.name] = rr

summary_rows: List[Dict[str, Any]] = []

# -------------------------
# 2) Evaluate each run on CLEAN
# -------------------------
for ds_dir in sorted(ds_dirs):
    ds_name = ds_dir.name
    if ds_name not in roots_map:
        print(f"⏭️  Skip OUT dataset not in roots: {ds_name}")
        continue

    ds_root = roots_map[ds_name]

    # build clean eval view once per dataset
    try:
        vroot_clean, clean_yaml, split_mode = build_clean_runtime_view_and_yaml(ds_root)
        print(f"\n[Dataset] {ds_name}")
        print(f" - ds_root      : {ds_root}")
        print(f" - clean_yaml   : {clean_yaml}")
        print(f" - split_mode   : {split_mode}")
    except Exception as e:
        print(f"\n[Dataset] {ds_name}  ❌ build clean view failed: {e}")
        continue

    # iterate run directories (trained cases)
    run_dirs = [p for p in ds_dir.iterdir() if p.is_dir()]
    if not run_dirs:
        print(f"  ⏭️  No runs under: {ds_dir}")
        if CLEANUP_RUNTIME_VROOT:
            safe_rmtree(vroot_clean)
        continue

    for run_dir in sorted(run_dirs):
        # prefer reading Cell2 metadata if exists
        meta_path = run_dir / "metrics_eval.json"
        meta: Dict[str, Any] = {}
        if meta_path.exists():
            try:
                meta = json.loads(meta_path.read_text(encoding="utf-8"))
            except Exception:
                meta = {}

        # parse fields (fallback to dir name pattern: model__case__classmode__nocopy)
        model_tag = meta.get("model_tag")
        case_tag  = meta.get("case_tag")
        class_mode = meta.get("class_mode")
        single_cls = bool(meta.get("single_cls", False))
        used_imgsz = meta.get("effective_imgsz") or IMG_SIZE

        if not (model_tag and case_tag and class_mode):
            parts = run_dir.name.split("__")
            if len(parts) >= 4:
                model_tag  = model_tag  or parts[0]
                case_tag   = case_tag   or parts[1]
                class_mode = class_mode or parts[2]
                single_cls = (class_mode == "object_only")
            else:
                # unknown naming
                model_tag  = model_tag  or "unknown"
                case_tag   = case_tag   or "unknown"
                class_mode = class_mode or "unknown"
                # keep single_cls as-is

        if not should_eval_case(str(case_tag)):
            continue

        wpath, wname = pick_weight_file(run_dir)
        if wpath is None:
            print(f"  ⏭️  Skip (no weights): {run_dir}")
            continue

        # run evaluation
        print(f"  [Eval] run={run_dir.name}")
        print(f"    - model_tag   : {model_tag}")
        print(f"    - case_tag    : {case_tag}")
        print(f"    - class_mode  : {class_mode} (single_cls={single_cls})")
        print(f"    - weight      : {wname}")
        print(f"    - imgsz       : {used_imgsz}")

        val_res = None
        metrics: Dict[str, Any] = {}
        ok = True
        err_msg = ""

        try:
            eval_model = YOLO(str(wpath))
            with suppress_output(SILENCE_ULTRA_OUTPUT):
                val_res = eval_model.val(
                    data=str(clean_yaml),
                    imgsz=int(used_imgsz) if used_imgsz else IMG_SIZE,
                    device=DEVICE,
                    split="val",
                    verbose=False,
                    workers=NUM_WORKERS,
                    single_cls=single_cls,
                )
            metrics = extract_metrics_dict(val_res)
        except Exception as e:
            ok = False
            err_msg = str(e)
            metrics = {}

        # save per-run metrics
        out_json = run_dir / "metrics_on_clean.json"
        try:
            with open(out_json, "w", encoding="utf-8") as f:
                json.dump(
                    {
                        "dataset": ds_name,
                        "ds_root": str(ds_root),
                        "run_dir": str(run_dir),
                        "model_tag": model_tag,
                        "case_tag": case_tag,
                        "class_mode": class_mode,
                        "single_cls": single_cls,
                        "weight_name": wname,
                        "weight_path": str(wpath),
                        "clean_yaml": str(clean_yaml),
                        "runtime_view_root": str(vroot_clean),
                        "split_mode": split_mode,
                        "imgsz": used_imgsz,
                        "ok": ok,
                        "error": err_msg,
                        "metrics": metrics,
                    },
                    f,
                    ensure_ascii=False,
                    indent=2,
                )
        except Exception:
            pass

        # summary row
        row = {
            "dataset": ds_name,
            "run_name": run_dir.name,
            "model": model_tag,
            "case": case_tag,
            "class_mode": class_mode,
            "single_cls": single_cls,
            "weight_pick": wname,
            "weight_path": str(wpath),
            "imgsz": used_imgsz,
            "ok": ok,
            "error": err_msg,
        }

        # pick common ultralytics keys if present (version-dependent)
        common_keys = [
            "metrics/mAP50(B)",
            "metrics/mAP50-95(B)",
            "metrics/precision(B)",
            "metrics/recall(B)",
            "fitness",
        ]
        for k in common_keys:
            if k in metrics:
                row[k] = metrics[k]

        # also keep any alternative keys sometimes used
        alt_keys = ["map50", "map", "mp", "mr", "box/map50", "box/map", "box/mp", "box/mr"]
        for k in alt_keys:
            if k in metrics:
                row[k] = metrics[k]

        summary_rows.append(row)

        # cleanup
        try:
            del eval_model
        except Exception:
            pass
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

    if CLEANUP_RUNTIME_VROOT:
        safe_rmtree(vroot_clean)

# -------------------------
# 3) Save summary CSV
# -------------------------
out_csv = OUT_ROOT / "summary_eval_refine_proposed(100).csv"

cols = set()
for r in summary_rows:
    cols.update(r.keys())

base_cols = [
    "dataset", "run_name", "model", "case",
    "class_mode", "single_cls",
    "weight_pick", "weight_path",
    "imgsz", "ok", "error",
    "metrics/mAP50(B)", "metrics/mAP50-95(B)", "metrics/precision(B)", "metrics/recall(B)",
    "fitness",
]
extra_cols = sorted([c for c in cols if c not in set(base_cols)])
final_cols = base_cols + extra_cols

with open(out_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=final_cols)
    w.writeheader()
    for r in summary_rows:
        w.writerow(r)

print("\n" + "=" * 80)
print(f"✅ Saved CLEAN-eval summary CSV: {out_csv}")
print(f"✅ Total eval runs: {len(summary_rows)}")
print("=" * 80)


[EVAL on CLEAN] Load Cell2 weights and evaluate on ds_root/labels (CLEAN)
 - OUT_ROOT               : /home/ISW/project/object_detect_refine_sam
 - RUNTIME_EVAL_VROOT_BASE : /home/ISW/project/_runtime_eval_views
 - DEVICE                 : 0
 - NUM_WORKERS            : 8
 - IMG_SIZE(default)      : 640
 - EVAL_ONLY_NOISE_CASES  : True (scale/side only)

[Dataset] BCCD
 - ds_root      : /home/ISW/project/datasets/BCCD
 - clean_yaml   : /home/ISW/project/_runtime_eval_views/BCCD/clean_eval_view/data.yaml
 - split_mode   : explicit
  [Eval] run=yolo11n__scale_0.6__multiclass__DIRECT
    - model_tag   : yolo11n
    - case_tag    : scale_0.6
    - class_mode  : multiclass (single_cls=False)
    - weight      : best.pt
    - imgsz       : 640
  [Eval] run=yolo11n__scale_0.8__multiclass__DIRECT
    - model_tag   : yolo11n
    - case_tag    : scale_0.8
    - class_mode  : multiclass (single_cls=False)
    - weight      : best.pt
    - imgsz       : 640
  [Eval] run=yolo11n__scale_1.2__multicla